# NVIDIA FLARE's Federated Computing Platform

In this chapter, we will overview the core concepts and system architecture of NVIDIA FLARE (NVFlare). We will explore different aspects of the NVFlare system, simulate deployment, and learn how to interact with the system.

## Federated Learning vs. Federated Computing

At its core, FLARE serves as a federated computing framework, with applications such as Federated Learning and Federated Analytics built upon this foundation. Notably, it is agnostic to datasets, workloads, and domains. Unlike centralized data lake solutions that require copying data to a central location, FLARE brings computing capabilities directly to distributed datasets. This approach ensures that data remains within the compute node, with only pre-approved, selected results shared among collaborators. Moreover, FLARE is system agnostic, offering easy integration with various data processing frameworks through the implementation of the FLARE client. This client facilitates deployment in sub-processes, Docker containers, Kubernetes pods, HPC, or specialized systems.



## Core Concepts

In NVIDIA FLARE (NVFlare), there are a few core concepts:

* Server-side component: Controller
* Client-side component: Executor
* Communication message: Shareable
* Filtering mechanism
* Building Block: FLComponent
* Job

In Part 1, we only encountered Job. We will discuss the rest in this section.

### Controller

The controller is the object that defines the logic for the clients to follow. The controller API makes it possible to create any client coordination logic in a federated learning workflow.

In other words, the controller defines the workflow: i.e., how the federated execution will be carried out. For example, whether the execution is in a round-robin style or scatter & gather style is defined by the controller.

The controller, in most cases, is executed on the FL server. Some refer to this as the server strategy. The controller can also be executed on the client side (referred to as client-side-controller). This can be used to define peer-to-peer styles of workflow, such as swarm learning.

### Executor

The Executor is the object that defines the logic to execute on the client side. It handles the tasks defined by the Controller and responds back to the task requests.

The interaction between the Controller and Executor can be found in the following picture:



### Shareable 

A [Shareable](https://nvflare.readthedocs.io/en/main/programming_guide/shareable.html) object represents communication between the server and client. Technically, a Shareable object is implemented as a Python dict. This dict contains two kinds of information:
* Header 
    * Peer Properties
    * Cookie 
    * Return code
* Content

In other words, a Shareable is nothing but a dictionary with some metadata information.


The Controller and Executor exchange Shareable

<img src="controller_executor_no_filter.png" alt="Controller and executor" width="700" height="400">



### Filters

NVIDIA FLARE also introduces a filtering mechanism to allow users to limit the input & outputs. Filters in NVIDIA FLARE are a way to transform the Shareable object between the communicating parties. A [Filter](https://nvflare.readthedocs.io/en/main/programming_guide/filters.html) can be used to provide additional processing to shareable data before sending or after receiving from the peer.

<img src="controller_worker_flow.png" alt="Controller and executor with filters" width="700" height="400">




### FLComponent

NVIDIA FLARE is built with components. FLComponent is the building block of all components (Base Class). Controller, Executor, Filter, and Shareable are all types of FLComponent.

The core property of FLComponent is event support. FLComponent is able to fire and receive events, enabling the FLARE system to be an event-driven, pluggable system.

### FLContext
One of the most important features of NVIDIA FLARE is nvflare.apis.fl_context, which is used to pass data between the FL components. FLContext is available to every method of all FLComponent types (Controller, Aggregator, Filter, Executor).


Through the FL Context, the component developer can:

* Get services provided by the underlying infrastructure

* Share data with other components of the FL system, even including components in the peer endpoints (between server and clients)

FLContext can be thought of as a Python dictionary that stores key/value pairs. Data items stored in FLContext are called properties, or props for short. Props have two attributes: visibility and stickiness.


### Events

NVIDIA FLARE fires and manages events in the lifecycle of the system. There are two categories of event types: Local Event and Fed Event.

Both client and server have local events for their respective system activities. The client's local event can also be converted to a "Fed Event," which means the event will propagate and fire on the server side.


## High-Level Concepts

Although understanding these core concepts will enable FLARE users to build powerful federated computing algorithms, some data scientists may prefer higher-level constructs.

NVFLARE also introduced a few concepts to reduce the learning curve.

 * FLModel -- higher-level communication data structure

### FLModel

The FLModel structure is a higher-level data structure designed for data scientists. This structure may not be general for common federated computing messaging communication, but it is suitable for federated learning applications.

We define a standard data structure, FLModel, that captures the common attributes needed for exchanging learning results. This is particularly useful when the NVFlare system needs to exchange learning information with external training scripts/systems. The external training script/system only needs to extract the required information from the received FLModel, run local training, and put the results in a new FLModel to be sent back.

Behind the scenes, we will convert the FLModel structure to and from Shareable.

**FLModel**

A standardized data structure for NVFlare to communicate with external systems.

**Parameters:**
* params_type – type of the parameters. It only describes the “params”. If params_type is None, params need to be None. If params are provided but params_type is not provided, then it will be treated as FULL.
* params – model parameters, for example, model weights for deep learning.
* optimizer_params – optimizer parameters. In many cases, the optimizer parameters don’t need to be transferred during FL training.
* metrics – evaluation metrics such as loss and scores.
* start_round – the start FL round. A round means a round trip between client/server during training. None for inference.
* current_round – the current FL round. A round means a round trip between client/server during training. None for inference.
* total_rounds – total number of FL rounds. A round means a round trip between client/server during training. None for inference.
* meta – metadata dictionary used to contain any key-value pairs to facilitate the process.

Now with a few concepts, let's take a look at the [system architecture](../03.1_federated_computing_architecture/system_architecture.ipynb).

